#문제1

4. GoogLeNet은 VGG19보다 적은 층으로 구성되어 있고 1x1 크기의 커널을 이용해 특성 개수를 줄여 VGG보다 성능이 더 좋다.

→ GoogLeNet은 22개의 층으로 구성, VGG19는 19층으로 형성되어 더 많은 층으로 구성된다.

#문제2

4. 합성곱 층을 1개 이상 사용한 인공 신경망을 합성곱 신경망이라고 한다.

→ 합성곱 신경망은 합성곱 연산을 사용하는 인공신경망이다.

#문제3

In [ ]:
#!pip install tensorflow-datasets
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

#load dataset
dataset_name = 'horses_or_humans'
(train_raw, val_raw, test_raw), info = tfds.load(name=dataset_name,
                    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                    with_info=True,
                    as_supervised=True,
                    shuffle_files=True,
                    batch_size = None)

#resize img
def img_resize(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    image = tf.image.resize(image, (224, 224))
    return image, label

train = train_raw.map(img_resize)
val = val_raw.map(img_resize)
test = test_raw.map(img_resize)


#model
model = keras.Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1000,activation="relu"))
model.add(Dense(units=1000,activation='softmax'))

model.summary()

#fit model
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy',metrics='accuracy')
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5',
                                               save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,
                                                 restore_best_weights=True)

history = model.fit(train, epochs=20,
                   validation_data = val,
                    batch_size=16,
                   callbacks=[checkpoint_cb, early_stopping_cb])

#fit result
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()
model.evaluate(test)

#문제4

5. CNN의 특징 추출 영역은 convolution layer과 pooling layer과 flatten layer로 구성된다.

→ flatten layer은 특징 추출 영역으로 구성되지 않는다.

#문제5

In [ ]:
# 구글 드라이브랑 연동
from google.colab import drive
drive.mount('/content/drive')

# 데이터 읽기
path = "/content/drive/MyDrive/RockPaperScissor"
train_dir = path+'/train-set'
test_dir = path+'/test-set'

In [ ]:
# 데이터 별 사진 분류 클래스
import os
classes = os.listdir(train_dir)

# 데이터 정리하기
import cv2
X_input = []
y_input = []
for i, cls in enumerate(os.listdir(train_dir)):
  for img in os.listdir(os.path.join(train_dir, cls)):
    image = cv2.imread(os.path.join(train_dir,cls,img), cv2.IMREAD_GRAYSCALE)
    X_input.append(image)
    y_input.append(i)

X_test = []
y_test = []
for i, cls in enumerate(os.listdir(test_dir)):
  for img in os.listdir(os.path.join(test_dir, cls)):
    image = cv2.imread(os.path.join(test_dir,cls,img), cv2.IMREAD_GRAYSCALE)
    X_test.append(image)
    y_test.append(i)

import numpy as np
X_test = np.array(X_test)
y_test = np.array(y_test)
X_input = np.array(X_input)
y_input = np.array(y_input)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 10, figsize=(20, 20))
for i in range(10):
  axs[i].imshow(X_input[i], cmap='gray_r')
  axs[i].axis('off')
plt.show()

In [ ]:
# train set와 validation set 나누기
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_input, y_input, test_size = 0.2, stratify= y_input)

# 이미지 전처리
X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

X_train = X_train.reshape(*(X_train.shape),1)
X_val = X_val.reshape(*(X_val.shape),1)
X_test = X_test.reshape(*(X_test.shape),1)

In [ ]:
# 합성곱 신경망 만들기
from tensorflow import keras
from tensorflow import keras
model = keras.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation = 'relu',padding = 'same', input_shape = (300,300,1)))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(128,(3,3),activation = 'relu',padding = 'same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(256, (3,3),activation = 'relu',padding = 'same'))
model.add(keras.layers.MaxPooling2D(2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(512,activation = 'relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(256, activation = 'relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(128, activation = 'relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(3, activation = 'softmax'))

# 층의 구성 그림으로 표현
keras.utils.plot_model(model)

In [ ]:
# 모델 컴파일과 훈련
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5',save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])